### Import necessary packages

In [1]:
import pandas as pd
import numpy as np
import snowflake.connector 
import yaml
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
pd.set_option('display.max_colwidth', 500)
# !pip install --upgrade snowflake-sqlalchemy
# !pip install pyyaml

In [ ]:
# https://www.markdownguide.org/cheat-sheet/

### just some resources
- [SNF sqlalchemy docs](https://docs.snowflake.com/en/user-guide/sqlalchemy.html)
- [reading & writing yamls](https://stackabuse.com/reading-and-writing-yaml-to-a-file-in-python/)
<br/> other yaml resources
- https://zetcode.com/python/yaml/
- https://www.cloudbees.com/blog/yaml-tutorial-everything-you-need-get-started/
- https://stackoverflow.com/questions/50472011/parsing-yaml-file-and-getting-a-dictionary 
<br/> yaml files need space after : to work.... 

You should store your credentials for snowflake somewhere not in your notebook.  It is highly discouraged to store any credentials in notebooks!
<br/>
[Here is some recommendations on where you can store credentials.](https://datarobot-public-api-client.readthedocs-hosted.com/en/v2.24.0/setup/getting_started.html#use-a-configuration-file)
- options are use a config file
- set credentials using environment variables (usually much easier to manage)

In [2]:
# open yaml file
with open(r'../PythonScripts/sf_config.yaml') as file:
    account_info = yaml.safe_load(file)

In [ ]:
# run line below to confirm your credentials but obviously don't show this to others
# account_info

In [3]:
engine1 = create_engine(URL(
    account = account_info['sf_account'],
    user = account_info['sf_user'],
    password = account_info['sf_password'],
    database = 'ELO_CLTV',
    schema = 'INFORMATION_SCHEMA',
    warehouse = 'PYTHON_WH',
    role='sysadmin',
))

engine2 = create_engine(URL(
    account = account_info['sf_account'],
    user = account_info['sf_user'],
    password = account_info['sf_password'],
    database = 'ELO_CLTV_CLONE',
    schema = 'INFORMATION_SCHEMA',
    warehouse = 'PYTHON_WH',
    role='sysadmin',
))

In [4]:
## obviously this can be more automated where you may want to have 1 specific warehouse/role that does this work
## and cycle through key value pairs of dbs you are comparing or have something generate 
## the key value pairs to compare
connection1 = engine1.connect()
connection2 = engine2.connect()

In [5]:
df1 = pd.read_sql_query("select * from tables", engine1)
df2 = pd.read_sql_query("select * from tables", engine2)

In [6]:
connection1.close()
engine1.dispose()
connection2.close()
engine2.dispose()

In [7]:
df1 = df1[df1['table_schema']=='PUBLIC']
slim_df1 = df1[['table_catalog','table_schema','table_name','row_count','created','last_altered']].sort_values(by=['table_name'])

In [8]:
slim_df1

,table_catalog,table_schema,table_name,row_count,created,last_altered
0,ELO_CLTV,PUBLIC,AGG_HIST_TRANSACTIONS,325540.0,2021-04-14 23:56:34.087000+00:00,2021-04-14 23:56:34.858000+00:00
1,ELO_CLTV,PUBLIC,AGG_NEW_MERCH_TRANSACTIONS,290001.0,2021-04-14 23:56:35.316000+00:00,2021-04-14 23:56:36.087000+00:00
2,ELO_CLTV,PUBLIC,CLONE_TEST,0.0,2021-04-07 14:22:11.141000+00:00,2021-04-07 14:22:11.261000+00:00
3,ELO_CLTV,PUBLIC,ELOCLTV_Predictions,0.0,2021-03-24 20:01:43.326000+00:00,2021-03-24 20:01:43.543000+00:00
4,ELO_CLTV,PUBLIC,ELO_CLTV_FINAL_DS_TRAIN,201917.0,2021-04-14 23:56:36.544000+00:00,2021-04-14 23:56:37.986000+00:00
5,ELO_CLTV,PUBLIC,ELO_CLTV_PREDS,123623.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:13.122000+00:00
6,ELO_CLTV,PUBLIC,HISTORICAL_TRANSACTIONS,29112361.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:06.610000+00:00
7,ELO_CLTV,PUBLIC,MERCHANTS,334696.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:08.501000+00:00
8,ELO_CLTV,PUBLIC,NEW_MERCHANT_TRANSACTIONS,1963031.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:09.858000+00:00
9,ELO_CLTV,PUBLIC,PREDS_TBL,50662230.0,2021-03-25 15:23:02.164000+00:00,2021-04-12 13:37:15.820000+00:00


In [9]:
df2 = df2[df2['table_schema']=='PUBLIC']
slim_df2 = df2[['table_catalog','table_schema','table_name','row_count','created','last_altered']].sort_values(by=['table_name'])

In [10]:
slim_df2 

,table_catalog,table_schema,table_name,row_count,created,last_altered
23,ELO_CLTV_CLONE,PUBLIC,AGG_HIST_TRANSACTIONS,325540.0,2021-04-14 23:58:49.751000+00:00,2021-04-14 23:58:50.399000+00:00
24,ELO_CLTV_CLONE,PUBLIC,AGG_NEW_MERCH_TRANSACTIONS,290001.0,2021-04-14 23:58:50.952000+00:00,2021-04-14 23:58:52.068000+00:00
25,ELO_CLTV_CLONE,PUBLIC,CLONE_TEST,0.0,2021-04-15 20:18:16.108000+00:00,2021-04-15 20:18:16.276000+00:00
26,ELO_CLTV_CLONE,PUBLIC,ELOCLTV_Predictions,0.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:21.076000+00:00
27,ELO_CLTV_CLONE,PUBLIC,ELO_CLTV_FINAL_DS_TRAIN,201917.0,2021-04-14 23:58:52.518000+00:00,2021-04-14 23:58:53.505000+00:00
28,ELO_CLTV_CLONE,PUBLIC,ELO_CLTV_PREDS,123623.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:23.092000+00:00
29,ELO_CLTV_CLONE,PUBLIC,FINAL_TABLE_FOR_SCORING,123623.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:24.157000+00:00
30,ELO_CLTV_CLONE,PUBLIC,HISTORICAL_TRANSACTIONS,29112361.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:15.858000+00:00
31,ELO_CLTV_CLONE,PUBLIC,MERCHANTS,334696.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:16.906000+00:00
32,ELO_CLTV_CLONE,PUBLIC,NEW_MERCHANT_TRANSACTIONS,1963031.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:17.615000+00:00


Thinking of ways to compare 2 pandas dataframes
- https://datatofish.com/compare-values-dataframes/

In [11]:
merged_df = slim_df1.merge(slim_df2, how='outer', on = 'table_name', indicator = True)
# merged_df
# suffix _x is the left ds, _y is the right ds (slim_df2 in this example)

In [12]:
merged_df

,table_catalog_x,table_schema_x,table_name,row_count_x,created_x,last_altered_x,table_catalog_y,table_schema_y,row_count_y,created_y,last_altered_y,_merge
0,ELO_CLTV,PUBLIC,AGG_HIST_TRANSACTIONS,325540.0,2021-04-14 23:56:34.087000+00:00,2021-04-14 23:56:34.858000+00:00,ELO_CLTV_CLONE,PUBLIC,325540.0,2021-04-14 23:58:49.751000+00:00,2021-04-14 23:58:50.399000+00:00,both
1,ELO_CLTV,PUBLIC,AGG_NEW_MERCH_TRANSACTIONS,290001.0,2021-04-14 23:56:35.316000+00:00,2021-04-14 23:56:36.087000+00:00,ELO_CLTV_CLONE,PUBLIC,290001.0,2021-04-14 23:58:50.952000+00:00,2021-04-14 23:58:52.068000+00:00,both
2,ELO_CLTV,PUBLIC,CLONE_TEST,0.0,2021-04-07 14:22:11.141000+00:00,2021-04-07 14:22:11.261000+00:00,ELO_CLTV_CLONE,PUBLIC,0.0,2021-04-15 20:18:16.108000+00:00,2021-04-15 20:18:16.276000+00:00,both
3,ELO_CLTV,PUBLIC,ELOCLTV_Predictions,0.0,2021-03-24 20:01:43.326000+00:00,2021-03-24 20:01:43.543000+00:00,ELO_CLTV_CLONE,PUBLIC,0.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:21.076000+00:00,both
4,ELO_CLTV,PUBLIC,ELO_CLTV_FINAL_DS_TRAIN,201917.0,2021-04-14 23:56:36.544000+00:00,2021-04-14 23:56:37.986000+00:00,ELO_CLTV_CLONE,PUBLIC,201917.0,2021-04-14 23:58:52.518000+00:00,2021-04-14 23:58:53.505000+00:00,both
5,ELO_CLTV,PUBLIC,ELO_CLTV_PREDS,123623.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:13.122000+00:00,ELO_CLTV_CLONE,PUBLIC,123623.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:23.092000+00:00,both
6,ELO_CLTV,PUBLIC,HISTORICAL_TRANSACTIONS,29112361.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:06.610000+00:00,ELO_CLTV_CLONE,PUBLIC,29112361.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:15.858000+00:00,both
7,ELO_CLTV,PUBLIC,MERCHANTS,334696.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:08.501000+00:00,ELO_CLTV_CLONE,PUBLIC,334696.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:16.906000+00:00,both
8,ELO_CLTV,PUBLIC,NEW_MERCHANT_TRANSACTIONS,1963031.0,2021-03-11 15:15:04.939000+00:00,2021-03-11 15:15:09.858000+00:00,ELO_CLTV_CLONE,PUBLIC,1963031.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:17.615000+00:00,both
9,ELO_CLTV,PUBLIC,PREDS_TBL,50662230.0,2021-03-25 15:23:02.164000+00:00,2021-04-12 13:37:15.820000+00:00,ELO_CLTV_CLONE,PUBLIC,50662230.0,2021-04-14 23:58:53.930000+00:00,2021-04-14 23:58:55.009000+00:00,both


In [ ]:
# merged_df.dtypes

In [14]:
def What_Now(dataframe_name):
    if dataframe_name['_merge'] == 'left_only':
        return "needs updating on right object"
    elif dataframe_name['_merge'] == 'right_only':
        return "needs updating on left object if development is finished"
    else: 
        if dataframe_name['last_altered_x'] > dataframe_name['last_altered_y'] :
            return "needs updating on right object"
        elif (dataframe_name['last_altered_x'] > dataframe_name['created_y']) & (dataframe_name['last_altered_x'] > dataframe_name['last_altered_y']):
            return "need to decide which object should be updated or if at all" # or send an email
        elif dataframe_name['last_altered_y'] > dataframe_name['last_altered_x']:
            return "needs updating on left object if development is finished"
            

In [15]:
merged_df['Next_step'] = [What_Now(row) for index, row in merged_df.iterrows()]

In [24]:
# fill in some Nans so other parts of the code will work
merged_df[['table_schema_x','table_schema_y']] = merged_df[['table_schema_x','table_schema_y']].fillna(value='PUBLIC')

# this works for this merged_df because first row has both table_catalog_x & _y 
# otherwise you'll want to find the row to fill it with
tab_cat_x = merged_df['table_catalog_x'][0]
tab_cat_y = merged_df['table_catalog_y'][0]

merged_df[['table_catalog_x']] = merged_df[['table_catalog_x']].fillna(tab_cat_x)
merged_df[['table_catalog_y']] = merged_df[['table_catalog_y']].fillna(tab_cat_y)

,table_catalog_x,table_schema_x,table_name,row_count_x,created_x,last_altered_x,table_catalog_y,table_schema_y,row_count_y,created_y,last_altered_y,_merge,Next_step
11,NaN,PUBLIC,FINAL_TABLE_FOR_SCORING,NaN,NaT,NaT,ELO_CLTV_CLONE,PUBLIC,123623.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:24.157000+00:00,right_only,needs updating on left object if development is finished
12,NaN,PUBLIC,TEST,NaN,NaT,NaT,ELO_CLTV_CLONE,PUBLIC,123623.0,2021-04-07 14:06:12.703000+00:00,2021-04-07 14:06:18.179000+00:00,right_only,needs updating on left object if development is finished


In [ ]:
## need to fill in nans for table_catalog & table schemas

In [ ]:
# now that we've identified which tables probably need to be updated 
# we can update the tables/objects that need to be updated

In [35]:
engine = create_engine(URL(
    account = account_info['sf_account'],
    user = account_info['sf_user'],
    password = account_info['sf_password'],
#     database = 'ELO_CLTV',
#     schema = 'INFORMATION_SCHEMA',
    warehouse = 'EDA_WH',
    role='sysadmin',
))

conn = engine.connect()

In [33]:
# Testing stuff out
for index, row in merged_df.iterrows():
    if row['Next_step'] == "needs updating on right object":
        obj = '"' + row['table_catalog_x'] + '"."' + row['table_schema_x'] + '"."' + row['table_name']+'"'
        print(obj)
        clone_obj = '"' + row['table_catalog_y'] + '"."' + row['table_schema_x'] + '"."' + row['table_name']+'"'
        print(clone_obj)
        string = "create or replace table " + clone_obj +" clone "+ obj
        print(string)

"ELO_CLTV"."PUBLIC"."CLONE_TEST"
"ELO_CLTV_CLONE"."PUBLIC"."CLONE_TEST"
create or replace table "ELO_CLTV_CLONE"."PUBLIC"."CLONE_TEST" clone "ELO_CLTV"."PUBLIC"."CLONE_TEST"


In [36]:
# this will execute your connections
for index, row in merged_df.iterrows():
    if row['Next_step'] == "needs updating on right object":
        obj = '"' + row['table_catalog_x'] + '".."' + row['table_name']+'"'
        clone_obj = '"' + row['table_catalog_y'] + '".."' + row['table_name']+'"'
        string = "create or replace table " + clone_obj +" clone "+ obj
        conn.execute(
            string
        )

In [37]:
# close things out once you're done
conn.close()
engine.dispose()